In [1]:
import numpy as np
import h5py
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pickle
import pandas as pd
_ = np.random.seed(0)

In [2]:
def ode_system(y, t, beta, kappa, gamma, N):
    S, E, I, R = y
    dSdt = -beta * S * I / N
    dEdt = beta * S * I / N - kappa * E
    dIdt = kappa * E - gamma * I
    dRdt = gamma * I
    return [dSdt, dEdt, dIdt, dRdt]

def simulate(parameters, ic, T):
    beta, kappa, gamma = parameters
    S0, E0, I0, R0 = ic

    N = S0 + E0 + I0 + R0
    y0=ic
    
    t = np.arange(T+1)
    ret = odeint(ode_system, y0, t, args=(beta, kappa, gamma, N))

    return kappa * ret[1:,1]

def poisson_noise(simulation):
    
    with_noise = np.random.poisson(np.maximum(simulation, 1e-6))

    return np.asarray(with_noise)

In [3]:
N  = 100000
E0 = 0
I0 = 10
R0 = 0
S0 = N - E0 - I0 - R0

init_cond = [S0, E0, I0, R0]

duration=100

In [4]:
with open('../sim_dataset.pkl', 'rb') as f:
    simulation_dataset = pickle.load(f)

In [5]:
def model(params):
    pvec = [
        params['beta'],
        params['kappa'],
        params['gamma']
    ]
    sim = simulate(pvec, init_cond, T=duration)
    
    return {'cases1': poisson_noise(sim)}

def distance(sim, obs):

    return np.linalg.norm(sim['cases1']-obs['cases1'])

In [6]:
import pyabc
from pyabc import ABCSMC, Distribution, RV, MultivariateNormalTransition, QuantileEpsilon
import tempfile

In [7]:
import pandas as pd
import numpy as np
import time
import pickle 

from sbi.utils import BoxUniform
from scipy.integrate import odeint
from sbi.utils import MultipleIndependent

import matplotlib.pyplot as plt

import torch
from torch.distributions import Uniform, Exponential, Cauchy

_ = torch.manual_seed(0)
_ = np.random.seed(0)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device) 

/etc/python/sitecustomize.py:236: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  mod = _original_import(name, globals, locals, fromlist, level)


Using device: cpu


In [8]:
import torch.nn as nn
from sbi.inference import NPE
from sbi.neural_nets import posterior_nn

In [9]:
low  = torch.tensor([0.2, 0.1, 0.05])
high = torch.tensor([1.5, 0.5, 0.4])

prior = BoxUniform(low=low, high=high)

In [10]:
class LSTMembedding(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=128, output_dim=30, num_layers=1,bidirectional=True):
        super().__init__()

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, bidirectional=bidirectional, batch_first = True)
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), output_dim)
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = x.unsqueeze(-1)  
        lstm_out, _ = self.lstm(x)  
        last_hidden = lstm_out[:, -1, :]
        last_hidden = self.dropout(last_hidden)
        out = self.fc(last_hidden)   
        return out

In [11]:
embedding_net =LSTMembedding(input_dim=1, hidden_dim=128, output_dim=30).to(device)
embedding_net

LSTMembedding(
  (lstm): LSTM(1, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=30, bias=True)
  (tanh): Tanh()
  (dropout): Dropout(p=0.1, inplace=False)
)

In [12]:
eps = QuantileEpsilon(initial_epsilon='from_sample', alpha=0.2)
transition = MultivariateNormalTransition(scaling=0.5)

In [13]:
prior = Distribution(
    beta=RV("uniform", 0.2, 1.5),
    kappa=RV("uniform", 0.1, 0.5),
    gamma=RV("uniform", 0.05, 0.4)
)

In [14]:
param_names = ["beta", "kappa","gamma"]

In [ ]:
pnpe_samples_1k=[]
gene_num = 10

for i, sim in enumerate(simulation_dataset):
    obs_data = sim['poisson']
    obs_dict={"cases1": obs_data[:,0]}
 
    db_path = "sqlite:///" + tempfile.mkstemp(suffix=f"abc_{i}.db")[1]

    abc = ABCSMC(model, prior, distance, population_size=100, transitions=transition, eps=eps)
    abc.new(db_path, obs_dict)

    start_time = time.time()
    history = abc.run(max_total_nr_simulations=500)

    df, weights = history.get_distribution()
    df = df[param_names]
    
    kde_estimator = pyabc.transition.MultivariateNormalTransition()
    kde_estimator.fit(df, weights)
    abc_samples = kde_estimator.rvs(500)

    x_obs = simulation_dataset[i]['poisson'][:,0]
    
    thetas = torch.tensor(np.array(abc_samples),dtype=torch.float32)

    xs = []
    for i, sample in abc_samples.iterrows():
        x_sim = poisson_noise(simulate(sample.values, init_cond, duration))
        xs.append(x_sim)
    xs = torch.tensor(xs, dtype=torch.float32)

    neural_posterior = posterior_nn(model='maf', embedding_net=embedding_net)
    inference = NPE(density_estimator=neural_posterior,device=device)
    
    density_estimator=inference.append_simulations(thetas, xs).train(training_batch_size=64)
    posterior = inference.build_posterior(density_estimator)
    end_time = time.time()

    samples = posterior.sample((10000,), x=x_obs)

    print(f"[{i}] Done in {end_time - start_time:.2f} seconds")

    samples_df=pd.DataFrame(samples.cpu().numpy())
    pnpe_samples_1k.append(samples)

ABC.Sampler INFO: Parallelize sampling on 128 processes.
ABC.History INFO: Start <ABCSMC id=1, start_time=2026-01-31 17:24:31>
ABC INFO: Calibration sample t = -1.
ABC INFO: t: 0, eps: 7.05289597e+03.
ABC INFO: Accepted: 100 / 895 = 1.1173e-01, ESS: 1.0000e+02.
ABC INFO: Stop: Total simulations budget.
ABC.History INFO: Done <ABCSMC id=1, duration=0:00:03.024111, end_time=2026-01-31 17:24:34>
/tmp/ipykernel_2361139/2356125261.py:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  xs = torch.tensor(xs, dtype=torch.float32)


 Training neural network. Epochs trained: 80

In [ ]:
with open("./posterior/PNPE_1k.pkl", "wb") as file:
    pickle.dump(pnpe_samples_1k, file)

In [ ]:
pnpe_samples_10k=[]
gene_num = 10

for i, sim in enumerate(simulation_dataset):
    obs_data = sim['poisson']
    obs_dict={"cases1": obs_data[:,0]}
 
    db_path = "sqlite:///" + tempfile.mkstemp(suffix=f"abc_{i}.db")[1]

    abc = ABCSMC(model, prior, distance, population_size=100, transitions=transition, eps=eps)
    abc.new(db_path, obs_dict)

    start_time = time.time()
    history = abc.run(max_total_nr_simulations=5000)

    df, weights = history.get_distribution()
    df = df[param_names]
    
    kde_estimator = pyabc.transition.MultivariateNormalTransition()
    kde_estimator.fit(df, weights)
    abc_samples = kde_estimator.rvs(5000)

    x_obs = simulation_dataset[i]['poisson'][:,0]
    
    thetas = torch.tensor(np.array(abc_samples),dtype=torch.float32)

    xs = []
    for i, sample in abc_samples.iterrows():
        x_sim = poisson_noise(simulate(sample.values, init_cond, duration))
        xs.append(x_sim)
    xs = torch.tensor(xs, dtype=torch.float32)

    neural_posterior = posterior_nn(model='maf', embedding_net=embedding_net)
    inference = NPE(density_estimator=neural_posterior,device=device)
    
    density_estimator=inference.append_simulations(thetas, xs).train(training_batch_size=128)
    posterior = inference.build_posterior(density_estimator)
    end_time = time.time()

    samples = posterior.sample((10000,), x=x_obs)

    print(f"[{i}] Done in {end_time - start_time:.2f} seconds")

    samples_df=pd.DataFrame(samples.cpu().numpy())
    pnpe_samples_10k.append(samples)

In [ ]:
with open("./posterior/PNPE_10k.pkl", "wb") as file:
    pickle.dump(pnpe_samples_10k, file)

In [ ]:
pnpe_samples_100k=[]
gene_num = 10

for i, sim in enumerate(simulation_dataset):
    obs_data = sim['poisson']
    obs_dict={"cases1": obs_data[:,0]}
 
    db_path = "sqlite:///" + tempfile.mkstemp(suffix=f"abc_{i}.db")[1]

    abc = ABCSMC(model, prior, distance, population_size=1000, transitions=transition, eps=eps)
    abc.new(db_path, obs_dict)

    start_time = time.time()
    history = abc.run(max_total_nr_simulations=50000)

    df, weights = history.get_distribution()
    df = df[param_names]
    
    kde_estimator = pyabc.transition.MultivariateNormalTransition()
    kde_estimator.fit(df, weights)
    abc_samples = kde_estimator.rvs(50000)

    x_obs = simulation_dataset[i]['poisson'][:,0]
    
    thetas = torch.tensor(np.array(abc_samples),dtype=torch.float32)

    xs = []
    for i, sample in abc_samples.iterrows():
        x_sim = poisson_noise(simulate(sample.values, init_cond, duration))
        xs.append(x_sim)
    xs = torch.tensor(xs, dtype=torch.float32)

    neural_posterior = posterior_nn(model='maf', embedding_net=embedding_net)
    inference = NPE(density_estimator=neural_posterior,device=device)
    
    density_estimator=inference.append_simulations(thetas, xs).train(training_batch_size=256)
    posterior = inference.build_posterior(density_estimator)
    end_time = time.time()

    samples = posterior.sample((10000,), x=x_obs)

    print(f"[{i}] Done in {end_time - start_time:.2f} seconds")

    samples_df=pd.DataFrame(samples.cpu().numpy())
    pnpe_samples_100k.append(samples)

In [ ]:
with open("./posterior/PNPE_100k.pkl", "wb") as file:
    pickle.dump(pnpe_samples_100k, file)